In [1]:
import os
os.chdir('../')

In [2]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *

Loading imports


In [3]:
torch.cuda.set_device(4)

## Load Model

In [4]:
name = 'baseline_any_xse_resnext_512_appian'

In [5]:
import pretrainedmodels

In [6]:
m = pretrainedmodels.se_resnext50_32x4d(num_classes=6, pretrained=None)

In [7]:
m.avg_pool = nn.AdaptiveAvgPool2d(1)

In [8]:
# state['model'].keys()

In [9]:
# sp2 = '~/kaggle/rsna_retro/models/model100/fold0_ep2.pt'
sp3 = '~/kaggle/rsna_retro/models/model100/fold0_ep3.pt'

saved_path = Path(sp3).expanduser()
state = torch.load(saved_path)
m.load_state_dict(state['model'])

<All keys matched successfully>

In [10]:
dbunch = get_data(48, 512, splits=Meta.splits, img_dir=path_jpg)
learn = get_learner(dbunch, m)

## Training

In [11]:
do_fit(learn, 2, 5e-4, freeze=True)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.069960,0.074325,0.976008,0.955013,3:13:11


epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.070235,0.071104,0.976525,0.956204,3:12:03
1,0.054933,0.066092,0.978258,0.958980,3:11:02


## Submission

In [ ]:
learn.load(f'runs/{name}-1')
sub_fn = f'subm/{name}'

In [ ]:
learn.dbunch = get_test_data(Meta.df_tst, bs=128, sz=None, tst_dir='tst_jpg')

In [ ]:
ob = learn.dbunch.one_batch()

In [ ]:
preds,targs = learn.get_preds()

In [15]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [16]:
api.competition_submit(f'{sub_fn}.csv', f'{name} appian se_resnext50', 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:02<00:00, 9.96MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [17]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 13820147,
 'totalBytes': 27277209,
 'date': '2019-12-16T19:19:33.217Z',
 'description': 'baseline_any_xse_resnext_512_appian appian se_resnext50',
 'errorDescription': None,
 'fileName': 'baseline_any_xse_resnext_512_appian.csv',
 'publicScore': None,
 'privateScore': None,
 'status': 'pending',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/13820147/13820147.raw'}

## Save Predictions

In [11]:
class FeatureModel(nn.Module):
    def __init__(self, m): 
        super().__init__()
        self.m = m
        
    def forward(self, x):
        return self.m.avg_pool(self.m.features(x))

In [12]:
#export
class DummyLoss:
    def __call__(self, p, *t, **kwargs): return torch.tensor(0, device=p.device).float()

In [13]:
#export
def save_features(learn, feat_path):
    preds,targs = learn.get_preds(dl=learn.dbunch.valid_dl)
    val_ids = dbunch.valid_dl.dataset.items
    feat_path.mkdir(exist_ok=True)
    for idx,pred in progress_bar(zip(val_ids, preds), total=len(val_ids)):
        fn = feat_path/f'{idx}'
        if fn.exists(): continue
        np.save(str(fn), pred.squeeze().numpy())

In [14]:
#export
path_feat512 = path/'features_512'
path_feat512_tst = path/'tst_features_512'

In [15]:
len(dbunch.valid_dl)

1306

In [16]:
dbunch = get_test_data(Meta.df_comb, bs=128, sz=None, tst_dir='nocrop_jpg', sl=slice(50000, 200000))
# dbunch = get_data(48, 512, splits=Meta.splits_sample, img_dir=path_jpg)
learn = get_learner(dbunch, m, lf=DummyLoss(), metrics=[])
learn.load(f'runs/{name}-1')
learn.model = FeatureModel(learn.model)

In [17]:
save_features(learn, path_feat512)

In [18]:
import gc
gc.collect()

20

In [ ]:
learn.dbunch = get_test_data(Meta.df_tst, bs=128, sz=None, tst_dir='tst_jpg')
save_features(learn, path_feat512_tst)